In [417]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This process will bring together the data sources so that they will be more easily queryable for our investigation later. the intention is to create two files. Each row in the these files is California school. each school will have the demographic information outlined in the proposal. the first file will contain the data for schools that for which migrant data is available. The Second file will contain this demographic information for every California school.

****

## Transformations

These dataframes will be created in parallel because the processes are nearly identicle. First remove any nulls, in this case some of the scores are starred out.



**1. Mean Score by School**
+ Mean Scale score is agregated using the groupby function to give an average score for each school 

**2. Entities**
+ the dataframe from the previous step is joined with information about the schools location that will be used to determine sanctuary status.

**3. Per Pupil Spending**
+ Data about the schools per pupil spending is joined via the district code

**4. Local Median Income**
+ Data about the median income in the schools zip code 

**5. Education Experience**
+ records of employees are agregated at the district level to give a metric for the average number of years experience a teacher has.

**6. Food Aid Utilization**
+ at the school level information about both the availability and actual utilization of free and reduced priced meals aimed at those struggling with food insecurity

**7. Staff Ethnicity**
+ for each school district the ratios of the 10 available ethnicities is calculated, on the district code 10 more columns are added to represent the frequency of the ethnic groups.



****
**1. Mean Score by School **

clean for non numeric inputs, then the goal is to create a list of all school code with the average mean scale score.

****

In [418]:
#create a data frame every test for every grade for every school
Test_Results = pd.read_csv('Data/ca2016_all.csv')
Test_Results = Test_Results[Test_Results['Mean Scale Score']!='*']
Test_Results['Mean Scale Score'] = pd.to_numeric(Test_Results['Mean Scale Score'])
Test_Results.iloc[1]

Unnamed: 0                             1613
County Code                               1
District Code                         10017
School Code                          112607
Test Year                              2016
Subgroup ID                               1
Test Type                                 B
Total CAASPP Enrollment                  91
Total Tested At Entity Level             89
Total Tested with Scores                 89
Grade                                    11
Test Id                                   2
CAASPP Reported Enrollment               91
Students Tested                          89
Mean Scale Score                     2506.3
Percentage Standard Exceeded              0
Percentage Standard Met                  11
Percentage Standard Met and Above        11
Percentage Standard Nearly Met           24
Percentage Standard Not Met              65
Students with Scores                     89
Area 1 Percentage Above Standard          2
Area 1 Percentage Near Standard 

In [419]:
Migrant_Test_Results = Test_Results[Test_Results['Subgroup ID']==28]
Migrant_School_List = Migrant_Test_Results.groupby('School Code')['Mean Scale Score'].mean().reset_index()
Migrant_School_List.head(3)
#here is basis on which the rest of the DataFrame will be built

,School Code,Mean Scale Score
0,100040,2520.025
1,100198,2515.700
2,100248,2391.600


In [420]:
#creating that same dataframe but for all schools
All_Test_Results = Test_Results[Test_Results['Subgroup ID']==1]
All_School_List = All_Test_Results.groupby('School Code')['Mean Scale Score'].mean().reset_index()
All_School_List.head(3)

,School Code,Mean Scale Score
0,100016,2497.525
1,100024,2520.000
2,100040,2537.900


As expected we have many more schools filled in for our all school list than our migrant school list

In [421]:
print("Migrant size: ",len(Migrant_School_List)," All size: ", len(All_School_List))

('Migrant size: ', 379, ' All size: ', 9217)


****
**2. Entities **

Join both list with thier location and district information using school code in the Entities file

****

In [422]:
Entities = pd.read_csv('Data/sb_ca2016entities.csv')
Migrant_School_List = Migrant_School_List.merge(Entities[['County Name','School Code','District Code','Zip Code','District Name']],on='School Code')
Migrant_School_List = Migrant_School_List[['Mean Scale Score','County Name','School Code','District Code','Zip Code','District Name']]
Migrant_School_List.head(2)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name
0,2520.025,Sacramento,100040,67348,95632,Galt Joint Union Elementary School Distr
1,2515.700,Kern,100198,63529,93307,Kern High School District


In [423]:
#now repeat this process for the dataframe with all schools
All_School_List = All_School_List.merge(Entities[['County Name','School Code','District Code','Zip Code','District Name']],on='School Code')
All_School_List = All_School_List[['Mean Scale Score','County Name','School Code','District Code','Zip Code','District Name']]
All_School_List.head(2)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name
0,2497.525,Madera,100016,65243,93638,Sherman Thomas Charter
1,2520.000,El Dorado,100024,61838,95762,Buckeye Union Elementary School District


****
**3. Per Pupil Spending **

Import the per pupil spending by district and merge it the existing DataFrame. this will also generate a column about the number of students who regularly attend that school

****

In [424]:
Per_Pupil_Spending = pd.read_csv('Data/currentexpense1617.csv')
Per_Pupil_Spending.head(1)
#merge each DataFrame on District code to give each school is district funding information

,﻿CO,District Code,DISTRICT,Total_Budget,Total_Students,Per_Student_Spending,LEA Type
0,1,61119,Alameda Unified,"104,846,431","9,061.8","11,570",Unified


In [425]:
Migrant_School_List = Migrant_School_List.merge(Per_Pupil_Spending[['District Code','Total_Budget','Total_Students','Per_Student_Spending']])
Migrant_School_List.head(2)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Budget,Total_Students,Per_Student_Spending
0,2520.0250,Sacramento,100040,67348,95632,Galt Joint Union Elementary School Distr,"39,533,735","3,455.2","11,442"
1,2419.7625,Sacramento,6033310,67348,95632,Galt Joint Union Elementary School Distr,"39,533,735","3,455.2","11,442"


In [426]:
All_School_List = All_School_List.merge(Per_Pupil_Spending[['District Code','Total_Budget','Total_Students','Per_Student_Spending']])
All_School_List.head(2)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Budget,Total_Students,Per_Student_Spending
0,2497.525,Madera,100016,65243,93638,Sherman Thomas Charter,"208,196,277","19,012.0","10,951"
1,2465.150,Madera,107938,65243,93638,Ezequiel Tafoya Alvarado Academy,"208,196,277","19,012.0","10,951"


****
**4. Local Median Income **

Similiar to the previous step we will load in extra data from a file and merge. The key difference here is that Median Income is merged on the Zip Code

****

In [427]:
Zipcode_Income = pd.read_csv('Data/zipcode_income.csv')
Zipcode_Income['Zip Code'] = Zipcode_Income['Zip Code'].astype(str)
Zipcode_Income.head(1)

,Zip Code,Median,Mean,Pop
0,1001,"56,663","66,688","16,445"


In [428]:
Migrant_School_List  = Migrant_School_List.merge(Zipcode_Income)
All_School_List  =All_School_List.merge(Zipcode_Income)
All_School_List.head(1)
#now both DataFrames have the Apropriate Income Data

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Budget,Total_Students,Per_Student_Spending,Median,Mean,Pop
0,2497.525,Madera,100016,65243,93638,Sherman Thomas Charter,"208,196,277","19,012.0","10,951","40,096","52,034","49,068"


****
**5. Educational Expereince **

The next file is a list of all current employees, what district they work in and how many years they have been a teacher. the average years will be created by grouping by District Code.

****

In [429]:
Staff_List = pd.read_csv('Data/Staff_Files.csv')
Staff_List['District Code'] =  Staff_List['District Code'].astype(str)
Staff_List.head(1)

,﻿AcademicYear,RecID,District Code,County Name,District Name,GenderCode,Age,EducationLevel,EthnicGroup,YearsTeaching,YearsInDistrict,EmploymentStatusCode,FTE Teaching,FTE administrative,FTE PupilServices,FileCreated
0,1617,1118544,1964329,LOS ANGELES,Bonita Unified,M,53.0,V,7.0,21.0,13.0,T,45.0,0.0,0.0,1/16/2018


In [430]:
District_Experience = Staff_List.groupby('District Name')['YearsTeaching'].mean().reset_index()
Migrant_School_List  = Migrant_School_List.merge(District_Experience)
All_School_List = All_School_List.merge(District_Experience)
Migrant_School_List.head(1)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Budget,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching
0,2446.35,Kern,6009690,63560,93307,Lamont Elementary,"31,556,128","2,958.4","10,667","35,780","43,139","82,658",11.230769


****
**6. Food Aid Utilization **

This data will be joined at the school level. This is a measure of how much food assistance is being used by the various schools

****

In [431]:
Food_Aid = pd.read_csv('Data/Food_Assistance.csv')
Food_Aid.columns = ['School Code','Enrolled','Utilized','Percent_Eligible']
Food_Aid = Food_Aid.rename(index=str, columns={"Enrolled (K-12)": "Enrolled", "Utilized (K-12)": "Utilized","Percent_Eligable Eligible FPRM (K-12)":"Percent_Eligible"})
Food_Aid['Percent_Utilized'] = Food_Aid['Utilized']/ Food_Aid['Enrolled']
Food_Aid.head(1)

,School Code,Enrolled,Utilized,Percent_Eligible,Percent_Utilized
0,112607,413,308,0.75,0.745763


In [432]:
Migrant_School_List = Migrant_School_List.merge(Food_Aid)
All_School_List = All_School_List.merge(Food_Aid)
All_School_List.head(2)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Budget,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized
0,2674.80,El Dorado,930081,61853,95762,El Dorado Union High,"69,882,466","6,285.3","11,118","114,420","135,015","38,797",16.346939,2457,190,0.08,0.077330
1,2652.25,El Dorado,936302,61853,95682,El Dorado Union High,"69,882,466","6,285.3","11,118","79,253","93,832","29,114",16.346939,1827,379,0.21,0.207444


****
**8. Sanctuary Status **

Using a list of sanctuary county search through and label each school in a sanctuary county as a 1

****

In [433]:
sanctuaries  = {'Alameda','Contra Costa','Los Angeles','Monterey','Napa','Riverside','Sacramento','San Bernardino','San Diego','San Francisco','San Mateo','Santa Clara','Santa Cruz','Sonoma'
}


In [434]:
#now we update this new sanctuary column based on whether the county is in our sanctuary list
All_School_List['Sanctuary'] = np.empty(len(All_School_List))
for i in range(len(All_School_List)):
  
    if All_School_List.ix[i,'County Name'] in sanctuaries:
        All_School_List.ix[i,'Sanctuary'] = 1
    else:
        All_School_List.ix[i,'Sanctuary'] = 0
    #print(school_list.ix[i,'sanctuary'])
All_School_List.head(1)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Budget,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized,Sanctuary
0,2674.8,El Dorado,930081,61853,95762,El Dorado Union High,"69,882,466","6,285.3","11,118","114,420","135,015","38,797",16.346939,2457,190,0.08,0.07733,0.0


In [435]:
#now we update this new sanctuary column based on whether the county is in our sanctuary list
Migrant_School_List['Sanctuary'] = np.empty(len(Migrant_School_List))
for i in range(len(Migrant_School_List)):
  
    if Migrant_School_List.ix[i,'County Name'] in sanctuaries:
        Migrant_School_List.ix[i,'Sanctuary'] = 1
    else:
        Migrant_School_List.ix[i,'Sanctuary'] = 0
    #print(school_list.ix[i,'sanctuary'])
Migrant_School_List.head(1)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Budget,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized,Sanctuary
0,2446.35,Kern,6009690,63560,93307,Lamont Elementary,"31,556,128","2,958.4","10,667","35,780","43,139","82,658",11.230769,625,601,0.96,0.9616,0.0


In [436]:
#so How many of our 300 Migrant Schools fall in a sanctuary County
print(Migrant_School_List['Sanctuary'].count())

93


****
**9. Staff Ethnicity **

This data will be joined at the school level. This is a measure of how much food assistance is being used by the various schools

****

In [437]:
# to finish, export the two new dataframes

In [438]:
Migrant_School_List.to_csv('Data/Migrant_School_List.csv')
All_School_List.to_csv('Data/All_School_List.csv')